In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import date
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


In [6]:
#List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

617

In [11]:
cities[0:15]

['touros',
 'cape town',
 'axim',
 'puerto ayora',
 'kaitangata',
 'hermanus',
 'rikitea',
 'pandan',
 'hilo',
 'mataura',
 'busselton',
 'avera',
 'ijaki',
 'carnarvon',
 'victor harbor']

In [12]:
# OpenWeatherMap API Key
api_key = weather_api_key

# Starting URL for Weather Map API Call
url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=486a3f0a342100df108bae3ce4d66f0e


In [13]:
#Initiate counter for city number below
number = 1

#Lists for holding required items
city_name = []
lat = []
lng = []
temp = []
humid = []
clouds = []
wind = []

#Loop through each city, extract required items, and print result
print('------------------------------')
print('Start of Records')
print('------------------------------')

for city in cities:
    try:
        city_data = (requests.get(url + '&q=' + city)).json()
        city_name.append(city_data['name'])
        lat.append(city_data['coord']['lat'])
        lng.append(city_data['coord']['lon'])
        temp.append(city_data['main']['temp'])
        humid.append(city_data['main']['humidity'])
        clouds.append(city_data['clouds']['all'])
        wind.append(city_data['wind']['speed'])
        print(f'City number {number} of {len(cities)} complete. | Added {city}')
        number = number + 1
    
    except KeyError:
        print(f'Missing data in city number {number} of {len(cities)}. | Skipping {city}')
        number = number + 1
        
print('------------------------------')
print('End of Records')
print('------------------------------')

------------------------------
Start of Records
------------------------------
City number 1 of 617 complete. | Added touros
City number 2 of 617 complete. | Added cape town
City number 3 of 617 complete. | Added axim
City number 4 of 617 complete. | Added puerto ayora
City number 5 of 617 complete. | Added kaitangata
City number 6 of 617 complete. | Added hermanus
City number 7 of 617 complete. | Added rikitea
City number 8 of 617 complete. | Added pandan
City number 9 of 617 complete. | Added hilo
City number 10 of 617 complete. | Added mataura
City number 11 of 617 complete. | Added busselton
City number 12 of 617 complete. | Added avera
Missing data in city number 13 of 617. | Skipping ijaki
City number 14 of 617 complete. | Added carnarvon
City number 15 of 617 complete. | Added victor harbor
City number 16 of 617 complete. | Added kruisfontein
City number 17 of 617 complete. | Added tuktoyaktuk
City number 18 of 617 complete. | Added cidreira
Missing data in city number 19 of 617